In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from state_cleaner import *
from datetime import datetime
import requests
import json

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
os.chdir('../..')
os.chdir('CovidMobile')
os.getcwd()

'/Users/samismalling/CovidMobile'

In [3]:
fips_to_county = pd.read_csv('./data/ZIP-COUNTY-FIPS_2017-06.csv')
full_data = pd.read_csv('./data/compiled_2020-05-22.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (104,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,139,140,143,144) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
parks = full_data[['FIPS','park']][full_data['park'].notna()]

In [5]:
fips = parks['FIPS'].unique()
len(fips)

1008

In [6]:
fips_to_zips = dict()
for fip in fips:
    fips_to_zips[int(fip)] = list(fips_to_county[fips_to_county['STCOUNTYFP']==fip]['ZIP'])

In [7]:
def get_weather(locationid, datasetid, begin_date, end_date, mytoken, base_url):
    token = {'token': mytoken}
    
    #passing as string instead of dict because NOAA API does not like percent encoding
    params = 'datasetid='+str(datasetid)+'&'+'locationid='+str(locationid)+'&'+'startdate='+str(begin_date)+'&'+'enddate='+str(end_date)+'&'+'limit=15'+'&datatypeid=TMIN,TMAX,PRCP'
    
    r = requests.get(base_url+params, headers=token)
    #print("Request status code: "+str(r.status_code))

    try:
        #results comes in json form. Convert to dataframe
        df = pd.DataFrame.from_dict(r.json()['results'])
        #print("Successfully retrieved "+str(len(df['station'].unique()))+" stations")
        dates = pd.to_datetime(df['date'])

        return df
    except:
        pass

In [8]:
def fip_already_exists(fip):
    #Checks to see if the fip data is already in the database
    if conn.noaaweather.records.count_documents({ 'FIPS': fip }, limit = 1) != 0:
        return True
    else:
        return False

In [9]:
def date_fip_already_exists(fip,date):
    #Checks to see if the fip data for a specific date is already in the database
    if conn.noaaweather.records.count_documents({ 'FIPS': fip , 'date':date+'T00:00:00'}, limit = 1) != 0:
        return True
    else:
        return False

In [8]:
import json
import pymongo
import time
from pprint import pprint

conn=pymongo.MongoClient()
db = conn.noaaweather
records = db.records
conn.list_database_names()

['admin', 'config', 'local']

In [20]:
empty_fips = [9001,9003,9005,9007,9009,9011,9013,9015,22075,22095,23001,23003,23005,23007,
              23009,23011,23017,23019,23025,23031,25001,25003,25005,25009,25011,25013,25015,
              25017,25021,25023,25025,25027,26025,39133,39141,39151,39153,39155,39165,39173,
              40017,40021,40027,40031,40037,40047,40049,40089,40099,40101,40109,40113,40119,
              40121,40125,40131,40143,40151,41003,41005,41007,41009,41011,41015,41017,41019,
              41027,41029,41033,41035,41039,41041,41043,41047,41051,41053,41057,41059,41065,
              41067,41071,42001,42003,42007,42011,42013,42017,42019,42025,42027,42029,42033,
              42041,42043,42045,42049,42051,42055,42069,42071,42073,42075,42077,42079,42081,
              42089,42091,42095,42101,42107,42111,42115,42123,42125]

In [14]:
empty_fips = []

In [ ]:
baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?'
mytoken = 'NVPWXeGxislWKqThiizoffNjnPtUtAcT'
databaseid = 'GHCND'
begindate = '2020-02-15'
enddate = '2020-05-17'

tic = time.perf_counter()
weather = pd.DataFrame()
for fip, codes in fips_to_zips.items():
    fip_weather = pd.DataFrame()
    if not fip_already_exists(fip) and fip not in empty_fips: #check to make sure not double counting data
        print(fip)
        for code in codes:
            try:
                newdf = get_weather('ZIP:'+str(code), databaseid,begindate,enddate,mytoken,baseurl)
                fip_weather = fip_weather.append(newdf)
            except:
                continue
    if not fip_weather.empty:
        #average across the county
        precip = fip_weather[fip_weather['datatype']=='PRCP'][['date','value']].rename(columns={'value':'precipitation'})
        min_temp = fip_weather[fip_weather['datatype']=='TMIN'][['date','value']].rename(columns={'value':'min_temp'})
        max_temp = fip_weather[fip_weather['datatype']=='TMAX'][['date','value']].rename(columns={'value':'max_temp'})
        w = pd.merge(precip, temp, how ='outer', on ='date') 
        avgs = pd.merge(w.groupby('date', as_index=False)['precipitation'].mean(),
                        w.groupby('date', as_index=False)['min_temp'].mean(),
                        w.groupby('date', as_index=False)['max_temp'].mean(),on='date')
        avgs['FIPS']=fip
        avgs = avgs.to_dict()

        #insert records into MongoDB
        a = [{key:value[index] for key,value in avgs.items()}
             for index in range(max(map(len,avgs.values())))]
        #only insert records which are not already there:
        for line in a:
            if not date_fip_already_exists(line['FIPS'],line['date']):
                records.insert_one(line)
        print(records.count())
    else:
        empty_fips.append(fip)

toc= time.perf_counter()
print(f"Run time was {toc - tic:0.4f} seconds")

18117


#### Practice- only with zip codes from FIP 1001:

In [17]:
zips = fips_to_zips[1001]
len(zips)

13

In [18]:
baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?'
mytoken = 'NVPWXeGxislWKqThiizoffNjnPtUtAcT'
databaseid = 'GHCND'
begindate = '2020-03-01'
enddate = '2020-05-01'
weather = pd.DataFrame()
for code in zips[0:3]:
    print(code)
    try:
        newdf = get_weather('ZIP:'+str(code),databaseid,begindate,enddate,mytoken,baseurl)
        newdf.insert(0,'ZIP',code)
        weather = weather.append(newdf)
    except:
        continue

36003
36006
36067


In [12]:
records.find_one()

{'FIPS': 1001,
 '_id': ObjectId('5eb444b4f987a45eeb9d9046'),
 'date': '2020-03-01T00:00:00',
 'precipitation': 0.0,
 'temp': 33.0}

In [16]:
baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?'
mytoken = 'NVPWXeGxislWKqThiizoffNjnPtUtAcT'
databaseid = 'GHCND'
begindate = '2020-05-01'
enddate = '2020-05-01'
locationid = 'ZIP:36003'
token = {'token': mytoken}
params = 'datasetid='+databaseid+'&locationid='+str(locationid)+'&startdate='+str(begindate)+'&enddate='+str(enddate)+'&limit=15'+'&datatypeid=TMAX,TMIN,PRCP'
r = requests.get(baseurl+params, headers=token)
print("Request status code: "+str(r.status_code))
#df = pd.DataFrame.from_dict(r.json()['results'])

KeyboardInterrupt: 

In [ ]:
newdf = get_weather('ZIP:36003',databaseid,begindate,enddate,mytoken,baseurl)

In [21]:
precip = weather[weather['datatype']=='PRCP'][['ZIP','date','value']].rename(columns={'value':'precipitation'})
temp = weather[weather['datatype']=='TOBS'][['date','value']].rename(columns={'value':'temp'})
w = pd.merge(precip, temp, how ='outer', on ='date') 
avgs = pd.merge(w.groupby('date', as_index=False)['precipitation'].mean(),
                w.groupby('date', as_index=False)['temp'].mean(), on='date')
a = avgs.to_dict()

In [22]:
ld = [{key:value[index] for key,value in a.items()}
         for index in range(max(map(len,a.values())))]

[{'date': '2020-03-01T00:00:00', 'precipitation': 0, 'temp': nan},
 {'date': '2020-03-02T00:00:00', 'precipitation': 127, 'temp': nan},
 {'date': '2020-03-03T00:00:00', 'precipitation': 592, 'temp': nan},
 {'date': '2020-03-04T00:00:00', 'precipitation': 483, 'temp': nan},
 {'date': '2020-03-05T00:00:00', 'precipitation': 376, 'temp': nan},
 {'date': '2020-03-06T00:00:00', 'precipitation': 0, 'temp': nan},
 {'date': '2020-03-07T00:00:00', 'precipitation': 0, 'temp': nan},
 {'date': '2020-03-08T00:00:00', 'precipitation': 0, 'temp': nan},
 {'date': '2020-03-09T00:00:00', 'precipitation': 0, 'temp': nan},
 {'date': '2020-03-10T00:00:00', 'precipitation': 81, 'temp': nan},
 {'date': '2020-03-11T00:00:00', 'precipitation': 3, 'temp': nan},
 {'date': '2020-03-12T00:00:00', 'precipitation': 0, 'temp': nan},
 {'date': '2020-03-13T00:00:00', 'precipitation': 0, 'temp': nan},
 {'date': '2020-03-14T00:00:00', 'precipitation': 0, 'temp': nan},
 {'date': '2020-03-15T00:00:00', 'precipitation': 0, 

In [24]:
for line in ld:
    if line['date'] 

{'date': '2020-03-01T00:00:00', 'precipitation': 0, 'temp': nan}
{'date': '2020-03-02T00:00:00', 'precipitation': 127, 'temp': nan}
{'date': '2020-03-03T00:00:00', 'precipitation': 592, 'temp': nan}
{'date': '2020-03-04T00:00:00', 'precipitation': 483, 'temp': nan}
{'date': '2020-03-05T00:00:00', 'precipitation': 376, 'temp': nan}
{'date': '2020-03-06T00:00:00', 'precipitation': 0, 'temp': nan}
{'date': '2020-03-07T00:00:00', 'precipitation': 0, 'temp': nan}
{'date': '2020-03-08T00:00:00', 'precipitation': 0, 'temp': nan}
{'date': '2020-03-09T00:00:00', 'precipitation': 0, 'temp': nan}
{'date': '2020-03-10T00:00:00', 'precipitation': 81, 'temp': nan}
{'date': '2020-03-11T00:00:00', 'precipitation': 3, 'temp': nan}
{'date': '2020-03-12T00:00:00', 'precipitation': 0, 'temp': nan}
{'date': '2020-03-13T00:00:00', 'precipitation': 0, 'temp': nan}
{'date': '2020-03-14T00:00:00', 'precipitation': 0, 'temp': nan}
{'date': '2020-03-15T00:00:00', 'precipitation': 0, 'temp': nan}
{'date': '2020-0

### Export data to CSV:

In [ ]:
weather_sample = list(records.find( {} ))

In [ ]:
docs = pd.DataFrame(columns=[])
for num, doc in enumerate(weather_sample):
    # convert ObjectId() to str 
    doc["_id"] = str(doc["_id"]) 

    # get document _id from dict 
    doc_id = doc["_id"]
    
    # create a Series obj from the MongoDB dict 
    series_obj = pd.Series( doc, name=doc_id ) 

    # append the MongoDB Series obj to the DataFrame obj 
    docs = docs.append( series_obj )

In [ ]:
del docs['_id']
docs.reset_index(inplace=False)
docs = docs.astype({'date':'str'})
docs.head()

In [ ]:
docs.to_csv('./SafeGraph_data/NOAA_weather_sample.csv')

#### Check whether weather from one ZIP is different across a county

In [ ]:
import scipy.stats as stats

In [ ]:
stats.f_oneway(weather['precipitation'][weather['ZIP'] == weather['ZIP'].unique()[0]],
               weather['precipitation'][weather['ZIP'] == weather['ZIP'].unique()[1]],
               weather['precipitation'][weather['ZIP'] == weather['ZIP'].unique()[2]])

In [ ]:
stats.f_oneway(weather['temp'][weather['ZIP'] == weather['ZIP'].unique()[0]].dropna(), 
             weather['temp'][weather['ZIP'] == weather['ZIP'].unique()[1]].dropna(),
             weather['temp'][weather['ZIP'] == weather['ZIP'].unique()[2]].dropna())

### Getting weather data using the noaa package from github:

In [ ]:
n = noaa.NOAA()

In [ ]:
forecast = n.get_observations(20001, 'US',start='2020-04-28',end='2020-04-30')
f = pd.DataFrame(forecast)
items = ['timestamp','textDescription','temperature','precipitationLast6Hours','relativeHumidity','cloudLayers']
f = f[items]
f.insert(0,'ZIP',zip_code) #add zip code 

In [ ]:
dc_weather = pd.DataFrame(columns = ['ZIP','timestamp','textDescription','temperature','precipitationLast6Hours','relativeHumidity','cloudLayers'])
dczips = [20001,20004, 20005, 20036, 20009, 20007, 20010, 20008, 20016]
for code in dczips:
    forecast = n.get_observations(code, 'US',start='2020-04-28',end='2020-04-30')
    for obs in forecast:
        time = obs['timestamp']
        desc = obs['textDescription']
        temp = obs['temperature']['value']
        precip = obs['precipitationLast6Hours']['value']
        hum = obs['relativeHumidity']['value']
        cloud = obs['cloudLayers'][0]['base']['value']
    
        row = [code,time,desc,temp,precip,hum,cloud]
        weather.loc[len(weather)] = row

In [ ]:
weather.to_csv('/Users/samismalling/Documents/mobility-report-data-extractor-master/CovidMobile/data/DC_weather_04_28-04_30.csv')

In [ ]:
weather = pd.DataFrame()
bad_zips = []
for zip_code in fips_to_county['ZIP'][0:200]:
    if zip_code not in bad_zips:
        print(zip_code)
        try:
            forecast = n.get_forecasts(zip_code, 'US',hourly = False)
            f = pd.DataFrame(forecast)
            f = f[['startTime','endTime','temperature','windSpeed','windDirection','shortForecast']]
            f.insert(0,'ZIP',zip_code) #add zip code 
            weather = pd.concat([weather, f], ignore_index = True)
        except:
            bad_zips.append(zip_code)
            continue